### Predicting formation enthalpies for solid solutions of Lanthanides Orthophosphates


In [8]:
import os
import sys
import inspect

print(sys.version)

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
grandparentdir = os.path.dirname(parentdir)

sys.path.insert(0, grandparentdir) 

3.9.6 (default, Nov 10 2023, 13:38:27) 
[Clang 15.0.0 (clang-1500.1.0.2.5)]


In [9]:
import read_data
import featureSpan
import lasso
import Utils
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import scipy.optimize
import pandas as pd
import seaborn as sns
import scipy.stats as ss
from sklearn.linear_model import Lasso
from itertools import combinations, product
import itertools
import math
import pandas as pd

## Xenotime + Monazite

#### Load data from files

In [10]:
list1M = read_data.readData("../../data/DATA_HE_monazite.dat")
print("Shape of List1M is {}".format(list1M.shape))
print(list1M)

Shape of List1M is (525, 4)
[[5.70000000e+01 5.80000000e+01 7.50000000e-01 1.29717527e-01]
 [5.70000000e+01 5.80000000e+01 6.25000000e-01 1.90974365e-01]
 [5.70000000e+01 5.80000000e+01 5.00000000e-01 1.92712564e-01]
 ...
 [7.00000000e+01 7.10000000e+01 5.00000000e-01 2.12444086e-02]
 [7.00000000e+01 7.10000000e+01 3.75000000e-01 2.22639048e-02]
 [7.00000000e+01 7.10000000e+01 2.50000000e-01 2.09098752e-02]]


In [11]:
list2M = read_data.readCSVData_simplified("../../data/Data_Ln-monazite.csv")
print("Shape of List2M is {}".format(list2M.shape))
print((list2M))

Shape of List2M is (15, 2)
[[139.1         77.61001595]
 [146.5         75.78168414]
 [150.1         74.20076494]
 [154.1         72.98174154]
 [157.4         71.90916336]
 [160.3         70.88675537]
 [163.1         70.00137871]
 [165.2         69.15236746]
 [164.5         68.41325589]
 [165.4         67.64549329]
 [167.3         67.05867559]
 [169.1         66.33683909]
 [170.8         65.60276254]
 [172.2         65.17192527]
 [171.          64.45110847]]


In [12]:
list1X = read_data.readData("../../data/DATA_HE_xenotime.dat")
print("Shape of List1X is {}".format(list1X.shape))
print(list1X)

Shape of List1X is (525, 4)
[[5.70000000e+01 5.80000000e+01 7.50000000e-01 2.31598225e-01]
 [5.70000000e+01 5.80000000e+01 6.25000000e-01 2.78958493e-01]
 [5.70000000e+01 5.80000000e+01 5.00000000e-01 2.97583757e-01]
 ...
 [7.00000000e+01 7.10000000e+01 5.00000000e-01 7.77590152e-02]
 [7.00000000e+01 7.10000000e+01 3.75000000e-01 7.09266168e-02]
 [7.00000000e+01 7.10000000e+01 2.50000000e-01 5.91211365e-02]]


In [13]:
list2X = read_data.readCSVData_simplified("../../data/Data_Ln-xenotime.csv", material = "xenotime")
print("Shape of List2X is {}".format(list2X.shape))
print((list2X))

Shape of List2X is (15, 2)
[[ 91.7         83.16086967]
 [ 99.73        81.11579575]
 [112.27        79.31176899]
 [120.19        77.89975607]
 [127.26        76.63526873]
 [137.16        75.42961529]
 [143.83        74.36126731]
 [149.42        73.33463097]
 [156.33        72.43898327]
 [163.          71.49567572]
 [169.12        70.75467386]
 [177.09        69.85074728]
 [183.38        68.92826151]
 [188.35        68.35674618]
 [194.41        67.43658042]]


#### Generated Elemental Training/Testing sets from loaded data

In [14]:
listXM, listYM = featureSpan.generateFeatures(list1M, list2M)
XM = np.asarray(listXM)
YM = np.asarray(listYM)
print(XM.shape)
print(YM.shape)

listXX, listYX = featureSpan.generateFeatures(list1X, list2X)
XX = np.asarray(listXX)
YX = np.asarray(listYX)
print(XM.shape)
print(YM.shape)

X = np.concatenate((XM, XX), axis=0)
Y = np.concatenate((YM, YX), axis=0)
print(X.shape)
print(Y.shape)

elemental_list = ["m", "1/m", "m^2", "(1/m)^2", "(1-m)", "1/(1-m)", "(1-m)^2", "(1/(1-m))^2", "[AD:Young]", "[AD:Volume]", "1/[AD:Young]", "1/[AD:Volume]", "[AD:Young]^2", "[AD:Volume]^2", "(1/[AD:Young])^2", "(1/[AD:Volume])^2", "[AM:Young]", "[AM:Volume]", "1/[AM:Young]", "1/[AM:Volume]","[AM:Young]^2", "[AM:Volume]^2", "(1/[AM:Young])^2", "(1/[AM:Volume]])^2"]
print(len(elemental_list))
print(elemental_list)
m, n = X.shape

elemental_features = []

for i in elemental_list:
    elemental_features.append("("+i+")")
        
print("The elemental features are: \n {}".format(elemental_features))

(525, 24)
(525,)
(525, 24)
(525,)
(1050, 24)
(1050,)
24
['m', '1/m', 'm^2', '(1/m)^2', '(1-m)', '1/(1-m)', '(1-m)^2', '(1/(1-m))^2', '[AD:Young]', '[AD:Volume]', '1/[AD:Young]', '1/[AD:Volume]', '[AD:Young]^2', '[AD:Volume]^2', '(1/[AD:Young])^2', '(1/[AD:Volume])^2', '[AM:Young]', '[AM:Volume]', '1/[AM:Young]', '1/[AM:Volume]', '[AM:Young]^2', '[AM:Volume]^2', '(1/[AM:Young])^2', '(1/[AM:Volume]])^2']
The elemental features are: 
 ['(m)', '(1/m)', '(m^2)', '((1/m)^2)', '((1-m))', '(1/(1-m))', '((1-m)^2)', '((1/(1-m))^2)', '([AD:Young])', '([AD:Volume])', '(1/[AD:Young])', '(1/[AD:Volume])', '([AD:Young]^2)', '([AD:Volume]^2)', '((1/[AD:Young])^2)', '((1/[AD:Volume])^2)', '([AM:Young])', '([AM:Volume])', '(1/[AM:Young])', '(1/[AM:Volume])', '([AM:Young]^2)', '([AM:Volume]^2)', '((1/[AM:Young])^2)', '((1/[AM:Volume]])^2)']


In [15]:
dfX = pd.DataFrame(data=X, columns=elemental_features)

new_features=[]
new_columns = []

for i in range(n):
    for j in range(i):
        new_features.append(elemental_features[i]+"*"+elemental_features[j])
        new_columns.append(dfX[elemental_features[i]].values * dfX[elemental_features[j]].values)


for i in range(n):
    for j in range(i):
        for k in range(j):
            new_features.append(elemental_features[i]+"*"+elemental_features[j]+"*"+elemental_features[k])
            new_columns.append(dfX[elemental_features[i]] * dfX[elemental_features[j]] * dfX[elemental_features[k]])
            

#new_columns = np.reshape(np.asarray(new_columns))
            
new_columns = np.asarray(new_columns)
dfX = pd.concat(
    [
        dfX,
        pd.DataFrame(
            new_columns.T, 
            index=dfX.index, 
            columns=new_features
        )
    ], axis=1
)

dfX.head()

dfX.std() == 0
dfX=dfX.loc[:, dfX.std() > 0]
dfX.head()
#
#for i in range(len(new_features)):
#    print(new_features[i])

,(m),(1/m),(m^2),((1/m)^2),((1-m)),(1/(1-m)),((1-m)^2),((1/(1-m))^2),([AD:Young]),([AD:Volume]),...,((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AD:Young]^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AD:Volume]^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*((1/[AD:Young])^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*((1/[AD:Volume])^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Young]),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Volume]),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*(1/[AM:Young]),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*(1/[AM:Volume]),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Young]^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Volume]^2)
0,0.750,1.333333,0.562500,1.777778,0.250,4.000000,0.062500,16.000000,3.7,0.914166,...,1.141309e-07,6.967065e-09,6.089707e-10,9.975848e-09,0.000001,6.393987e-07,5.838102e-11,1.086996e-10,0.00017,0.000049
1,0.625,1.600000,0.390625,2.560000,0.375,2.666667,0.140625,7.111111,3.7,0.914166,...,1.141309e-07,6.967065e-09,6.089707e-10,9.975848e-09,0.000001,6.393987e-07,5.838102e-11,1.086996e-10,0.00017,0.000049
2,0.500,2.000000,0.250000,4.000000,0.500,2.000000,0.250000,4.000000,3.7,0.914166,...,1.141309e-07,6.967065e-09,6.089707e-10,9.975848e-09,0.000001,6.393987e-07,5.838102e-11,1.086996e-10,0.00017,0.000049
3,0.375,2.666667,0.140625,7.111111,0.625,1.600000,0.390625,2.560000,3.7,0.914166,...,1.141309e-07,6.967065e-09,6.089707e-10,9.975848e-09,0.000001,6.393987e-07,5.838102e-11,1.086996e-10,0.00017,0.000049
4,0.250,4.000000,0.062500,16.000000,0.750,1.333333,0.562500,1.777778,3.7,0.914166,...,1.141309e-07,6.967065e-09,6.089707e-10,9.975848e-09,0.000001,6.393987e-07,5.838102e-11,1.086996e-10,0.00017,0.000049


In [16]:
print(len(dfX.columns.values))
np.array(dfX.columns.values)

2320


array(['(m)', '(1/m)', '(m^2)', ...,
       '((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*(1/[AM:Volume])',
       '((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Young]^2)',
       '((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Volume]^2)'],
      dtype=object)

fitting of Lasso with a given $\lambda$

In [17]:
def LassoFit(lmb, X, Y, max_iter=100000, standardization = True):
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_standardized = scaler.transform(X)
    lasso =  Lasso(alpha=lmb, max_iter=max_iter)
    lasso.fit(X_standardized, Y.copy())
    coef =  lasso.coef_
    selected_indices = coef.nonzero()[0]
    selected_features = np.array(dfX.columns.values)[selected_indices]
    Y_predict = lasso.predict(X_standardized)
    MAE, MSE, ME = Utils.compute_error(Y.copy(), Y_predict)
        
    return coef, selected_indices, selected_features, MAE, MSE, ME

LassoFit(0.01, dfX, Y)

(array([ 0., -0.,  0., ...,  0., -0.,  0.]),
 array([ 282,  286,  386,  412,  531,  533,  547,  553,  555,  558,  570,
         588,  599,  651,  900, 1309, 1427, 1480, 1903, 2103, 2111, 2229]),
 array(['((1/[AM:Volume]])^2)*([AD:Volume])',
        '((1/[AM:Volume]])^2)*([AD:Volume]^2)',
        '([AD:Volume])*((1-m))*(m)', '([AD:Volume])*([AD:Young])*((1-m))',
        '([AD:Young]^2)*((1-m)^2)*(m)', '([AD:Young]^2)*((1-m)^2)*(m^2)',
        '([AD:Young]^2)*([AD:Young])*((1/m)^2)',
        '([AD:Young]^2)*([AD:Volume])*(1/m)',
        '([AD:Young]^2)*([AD:Volume])*((1/m)^2)',
        '([AD:Young]^2)*([AD:Volume])*((1-m)^2)',
        '([AD:Young]^2)*(1/[AD:Young])*([AD:Volume])',
        '([AD:Volume]^2)*((1-m))*(m)', '([AD:Volume]^2)*((1-m)^2)*(m^2)',
        '([AD:Volume]^2)*([AD:Young]^2)*((1/m)^2)',
        '([AM:Young])*([AD:Volume])*([AD:Young])',
        '(1/[AM:Volume])*([AD:Volume])*([AD:Young])',
        '(1/[AM:Volume])*(1/[AM:Young])*([AD:Volume])',
        '([AM:Young]^2)*(

#### Analyze the behavior of Lasso with the decrease of $\lambda$

In [18]:
#### Define a function which fits Lasso to have no more nonzero coefficients than a given threshold 
def LassoSelect(X, Y, min, max, step, threshold, standardization = True):
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_standardized = scaler.transform(X)
        
    found = False
    for lmbda in np.arange (min, max, step):
        coef, selected_indices, selected_features, MAE, MSE, ME = LassoFit(lmbda, X.copy(), Y.copy())
        if len(selected_indices) <= threshold:
            found = True
            break
    
    if found:
        print("FOUND with threshold: {}".format(threshold))
        print("Lambda: {}, nnz: {}, MAE: {}, MSE: {}, MAPE: {}".format(lmbda, len(selected_indices), MAE, MSE, ME))
            
    else:
        print("NOT FOUND with threshold: {}".format(threshold))
        print("Closest are: ")
        print("Lambda: {}, nnz: {}, MAE: {}, MSE: {}, MAPE: {}".format(lmbda, len(selected_indices), MAE, MSE, ME))
     
    X_reduced = X[selected_features]
    
    return X_reduced


X_reduced = LassoSelect(dfX, Y, 0.001, 0.101, 0.005, 10)
X_reduced.head()

FOUND with threshold: 10
Lambda: 0.026000000000000002, nnz: 9, MAE: 0.05862056871594235, MSE: 0.008455857119426544, MAPE: 0.08534564750261894


,([AD:Volume])*((1-m))*(m),([AD:Volume])*([AD:Young])*((1-m)),([AD:Young]^2)*((1-m)^2)*(m),([AD:Young]^2)*((1-m)^2)*(m^2),([AD:Young]^2)*([AD:Volume])*((1-m)),([AD:Volume]^2)*((1-m))*(m),([AD:Volume]^2)*((1-m)^2)*(m^2),([AM:Young]^2)*([AD:Volume])*([AD:Young]),((1/[AM:Volume]])^2)*([AD:Volume])*([AD:Young])
0,0.171406,0.845603,0.641719,0.481289,3.128733,0.156694,0.029380,68973.641885,0.000575
1,0.214258,1.268405,1.203223,0.752014,4.693099,0.195867,0.045906,68973.641885,0.000575
2,0.228541,1.691207,1.711250,0.855625,6.257466,0.208925,0.052231,68973.641885,0.000575
3,0.214258,2.114009,2.005371,0.752014,7.821832,0.195867,0.045906,68973.641885,0.000575
4,0.171406,2.536810,1.925156,0.481289,9.386198,0.156694,0.029380,68973.641885,0.000575


#### Lasso with $\ell_0$

In [19]:
def LassoL0(X, Y, nnz):    
    nr, nc = X.shape
    X = np.column_stack((X, np.ones(nr)))
    se_min = np.inner(Y, Y)
    coef_min, permu_min = None, None
    for permu in combinations(range(nc), nnz):
        X_ls = X[:, permu + (-1,)]
        coef, se, __1, __2 = np.linalg.lstsq(X_ls, Y, rcond=-1)
        try:
            if se[0] < se_min: 
                se_min = se[0]
                coef_min, permu_min = coef, permu
        except:
            pass
        
    return coef_min, permu_min

In [20]:
def LassoL0Fit(X_reduced, Y, nnz, log=True):
    
    scaler = StandardScaler()
    scaler.fit(X_reduced)
    X_std = scaler.transform(X_reduced)
    
    nr, nc = X_reduced.shape
   
    coefficients, selected_indices = LassoL0(X_std, Y, nnz)
   
    coefficients = np.array(coefficients)
    selected_indices = np.array(selected_indices)
    feature_reduced = np.array(X_reduced.columns.values)
    feature_list_selected = feature_reduced[selected_indices]
    
    X_selected = X_reduced[feature_list_selected]

    mean_selected = X_selected.mean()
    std_selected = X_selected.std()

    
    if log:
        print("Lasso: selected coefficients are: {}".format(coefficients))
        print("Lasso: selected features are: {}".format(feature_list_selected))
        
    #-mean/std
    mean_std = []
    for i in range(len(selected_indices)):
        mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
 
    sum_mean_std = sum(mean_std)

    for i in range(len(selected_indices)):
        coefficients[i] = coefficients[i] / std_selected[i]

    
    coefficients[len(selected_indices)] -= sum_mean_std
    
    function = str(coefficients[0])+" * "+feature_list_selected[0]
    
    for i in range(1, len(selected_indices)):
        if coefficients[i] >= 0:
            function += " + " + str(coefficients[i])+" * "+feature_list_selected[i]
        else:
            function += " - " + str(abs(coefficients[i]))+" * "+feature_list_selected[i]

    
    if coefficients[len(selected_indices)] >= 0:
        function += " + " + str(coefficients[len(selected_indices)])
    else:
        function += " - " + str(abs(coefficients[len(selected_indices)]))
    
    if log:
        print("Constructed function is: {}".format(function))

    X_selected = np.column_stack((X_selected, np.ones(X_selected.shape[0])))
    Y_predict = X_selected[:,0] * coefficients[0]

    for i in range(1,len(selected_indices)+1):
        Y_predict = Y_predict + X_selected[:,i] * coefficients[i]
    
    if log:
        Utils.print_error(Y.copy(),Y_predict,"Lasso L0: {} coef".format(nnz))
    
    return Y_predict, coefficients, selected_indices

In [21]:
LassoL0Fit(X_reduced, Y.copy(), 1);

Lasso: selected coefficients are: [3.00864236 2.41207633]
Lasso: selected features are: ['([AD:Volume]^2)*((1-m))*(m)']
Constructed function is: 1.1453165445832054 * ([AD:Volume]^2)*((1-m))*(m) - 0.05170210733884595
Lasso L0: 1 coef
Mean absolute error: 0.33165684084978003
Mean squared error: 0.2804752303920145
Mean absolute percentage error: 0.3120028984014222


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_19680/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_19680/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [22]:
LassoL0Fit(X_reduced, Y.copy(), 2);

Lasso: selected coefficients are: [1.77183769 1.37747441 2.41207633]
Lasso: selected features are: ['([AD:Volume]^2)*((1-m)^2)*(m^2)'
 '((1/[AM:Volume]])^2)*([AD:Volume])*([AD:Young])']
Constructed function is: 2.9453749063193713 * ([AD:Volume]^2)*((1-m)^2)*(m^2) + 108.10794470913883 * ((1/[AM:Volume]])^2)*([AD:Volume])*([AD:Young]) + 0.0429884502253266
Lasso L0: 2 coef
Mean absolute error: 0.09977744899236145
Mean squared error: 0.02625179680662067
Mean absolute percentage error: 0.26523613608204005


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_19680/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_19680/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [23]:
LassoL0Fit(X_reduced, Y.copy(), 3);

Lasso: selected coefficients are: [0.37965185 2.09193235 0.69287055 2.41207633]
Lasso: selected features are: ['([AD:Young]^2)*((1-m)^2)*(m)' '([AD:Volume]^2)*((1-m))*(m)'
 '((1/[AM:Volume]])^2)*([AD:Volume])*([AD:Young])']
Constructed function is: 0.0066205094852153186 * ([AD:Young]^2)*((1-m)^2)*(m) + 0.796347468412843 * ([AD:Volume]^2)*((1-m))*(m) + 54.37836859589801 * ((1/[AM:Volume]])^2)*([AD:Volume])*([AD:Young]) + 0.006479935228445477
Lasso L0: 3 coef
Mean absolute error: 0.08010324137767677
Mean squared error: 0.017334159698607292
Mean absolute percentage error: 0.09718154421984418


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_19680/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_19680/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [24]:
LassoL0Fit(X_reduced, Y.copy(), 4);

Lasso: selected coefficients are: [0.26960209 1.22738479 0.7666129  0.90307229 2.41207633]
Lasso: selected features are: ['([AD:Young]^2)*((1-m)^2)*(m)' '([AD:Volume]^2)*((1-m))*(m)'
 '([AD:Volume]^2)*((1-m)^2)*(m^2)'
 '((1/[AM:Volume]])^2)*([AD:Volume])*([AD:Young])']
Constructed function is: 0.004701421049791393 * ([AD:Young]^2)*((1-m)^2)*(m) + 0.46723536449379344 * ([AD:Volume]^2)*((1-m))*(m) + 1.2743618801822052 * ([AD:Volume]^2)*((1-m)^2)*(m^2) + 70.87557403864253 * ((1/[AM:Volume]])^2)*([AD:Volume])*([AD:Young]) + 0.019505964017146216
Lasso L0: 4 coef
Mean absolute error: 0.0630268184493695
Mean squared error: 0.009105184435617772
Mean absolute percentage error: 0.1467975518708706


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_19680/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_19680/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [25]:
LassoL0Fit(X_reduced, Y.copy(), 5);

Lasso: selected coefficients are: [0.20070196 0.2467823  0.75202455 0.98631679 0.99635434 2.41207633]
Lasso: selected features are: ['([AD:Volume])*((1-m))*(m)' '([AD:Young]^2)*([AD:Volume])*((1-m))'
 '([AD:Volume]^2)*((1-m))*(m)' '([AD:Volume]^2)*((1-m)^2)*(m^2)'
 '((1/[AM:Volume]])^2)*([AD:Volume])*([AD:Young])']
Constructed function is: 0.5117595826634067 * ([AD:Volume])*((1-m))*(m) + 0.0001474515072574851 * ([AD:Young]^2)*([AD:Volume])*((1-m)) + 0.2862773490180805 * ([AD:Volume]^2)*((1-m))*(m) + 1.6395817423024475 * ([AD:Volume]^2)*((1-m)^2)*(m^2) + 78.19660306717837 * ((1/[AM:Volume]])^2)*([AD:Volume])*([AD:Young]) - 0.07646874572108953
Lasso L0: 5 coef
Mean absolute error: 0.05947131800608015
Mean squared error: 0.008362937055313602
Mean absolute percentage error: 0.1770670074940963


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_19680/1248470983.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_19680/1248470983.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]
